In [83]:
import pandas as pd
from textblob import TextBlob
import nltk

#### Load datasets

In [84]:
news_data = pd.read_csv('../data/raw_analyst_ratings.csv')
stock_data = pd.read_csv('../data/GOOG_historical_data.csv')

#### Convert date columns to datetime

In [85]:
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')
stock_data['Date'] = pd.to_datetime(stock_data['Date'], errors='coerce')

#### Filter news for Google (ticker 'GOOG')

In [112]:
google_news = news_data[news_data['stock'] == 'GOOG'].copy()  
google_news

,Unnamed: 0,headline,url,publisher,date,stock
565380,568379,"Facebook, Snap Analyst Projects Q2 Revenue Ups...",https://www.benzinga.com/analyst-ratings/analy...,Shanthi Rexaline,2020-06-10 15:25:13-04:00,GOOG
565381,568380,How Cannabis Company Cannaphyll Is Marketing O...,https://www.benzinga.com/markets/cannabis/20/0...,Jaycee Tenn,2020-06-10 13:18:50-04:00,GOOG
565382,568381,"Twitter, Square Will Mark Juneteenth As Holida...",https://www.benzinga.com/news/20/06/16218441/t...,Shivdeep Dhaliwal,2020-06-10 00:28:00-04:00,GOOG
565383,568382,Price Over Earnings Overview: Alphabet,https://www.benzinga.com/intraday-update/20/06...,Benzinga Insights,2020-06-09 14:39:26-04:00,GOOG
565384,568383,Google Maps To Offer Relevant Local COVID-19 I...,https://www.benzinga.com/news/20/06/16208274/g...,Shivdeep Dhaliwal,2020-06-09 03:51:41-04:00,GOOG
...,...,...,...,...,...,...
566574,569584,Loup's Munster On Alphabet/Google Notes Short-...,https://www.benzinga.com/analyst-ratings/analy...,Benzinga Newsdesk,NaT,GOOG
566575,569585,"Moderate Inflation, Positive Retail Earnings O...",https://www.benzinga.com/news/earnings/18/11/1...,JJ Kinahan,NaT,GOOG
566576,569586,UPDATE: JANA Partners 13F Shows Fund Liquidate...,https://www.benzinga.com/news/18/11/12696718/u...,Benzinga Newsdesk,NaT,GOOG
566577,569587,Startup Point Of Sale Operating System Poynt R...,https://www.benzinga.com/fintech/18/11/1268071...,Spencer White,NaT,GOOG


#### Aggregate news headlines by date

In [87]:
google_news['date'] = google_news['date'].dt.date  
news_daily = google_news.groupby('date').size().reset_index(name='news_count')
google_news.head()

,Unnamed: 0,headline,url,publisher,date,stock
565380,568379,"Facebook, Snap Analyst Projects Q2 Revenue Ups...",https://www.benzinga.com/analyst-ratings/analy...,Shanthi Rexaline,2020-06-10,GOOG
565381,568380,How Cannabis Company Cannaphyll Is Marketing O...,https://www.benzinga.com/markets/cannabis/20/0...,Jaycee Tenn,2020-06-10,GOOG
565382,568381,"Twitter, Square Will Mark Juneteenth As Holida...",https://www.benzinga.com/news/20/06/16218441/t...,Shivdeep Dhaliwal,2020-06-10,GOOG
565383,568382,Price Over Earnings Overview: Alphabet,https://www.benzinga.com/intraday-update/20/06...,Benzinga Insights,2020-06-09,GOOG
565384,568383,Google Maps To Offer Relevant Local COVID-19 I...,https://www.benzinga.com/news/20/06/16208274/g...,Shivdeep Dhaliwal,2020-06-09,GOOG


In [88]:
news_daily.head()

,date,news_count
0,2020-06-04,1
1,2020-06-05,3
2,2020-06-08,1
3,2020-06-09,2
4,2020-06-10,3


#### Perform Sentiment Analysis

In [89]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [90]:
# Perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Apply sentiment analysis to each headline
google_news['sentiment'] = google_news['headline'].apply(get_sentiment)
google_news.head()

,Unnamed: 0,headline,url,publisher,date,stock,sentiment
565380,568379,"Facebook, Snap Analyst Projects Q2 Revenue Ups...",https://www.benzinga.com/analyst-ratings/analy...,Shanthi Rexaline,2020-06-10,GOOG,0.033333
565381,568380,How Cannabis Company Cannaphyll Is Marketing O...,https://www.benzinga.com/markets/cannabis/20/0...,Jaycee Tenn,2020-06-10,GOOG,0.000000
565382,568381,"Twitter, Square Will Mark Juneteenth As Holida...",https://www.benzinga.com/news/20/06/16218441/t...,Shivdeep Dhaliwal,2020-06-10,GOOG,0.000000
565383,568382,Price Over Earnings Overview: Alphabet,https://www.benzinga.com/intraday-update/20/06...,Benzinga Insights,2020-06-09,GOOG,0.000000
565384,568383,Google Maps To Offer Relevant Local COVID-19 I...,https://www.benzinga.com/news/20/06/16208274/g...,Shivdeep Dhaliwal,2020-06-09,GOOG,0.133333


In [91]:
# Aggregate sentiment scores by date
daily_sentiment = google_news.groupby('date')['sentiment'].mean().reset_index(name='avg_sentiment')
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'], errors='coerce')
daily_sentiment.head()

,date,avg_sentiment
0,2020-06-04,0.000000
1,2020-06-05,-0.051852
2,2020-06-08,0.139394
3,2020-06-09,0.066667
4,2020-06-10,0.011111


#### Calculate Stock Movements

In [92]:
# Calculate daily returns
stock_data['daily_return'] = stock_data['Close'].pct_change() * 100  # Percentage change
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,daily_return
0,2004-08-19,2.490664,2.591785,2.390042,2.499133,2.496292,897427216,0.0,0.0,NaN
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.694573,458857488,0.0,0.0,7.942989
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.721690,366857939,0.0,0.0,1.006362
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.608991,306396159,0.0,0.0,-4.140766
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.637103,184645512,0.0,0.0,1.077510


In [101]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'], errors='coerce')
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5020 entries, 0 to 5019
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5020 non-null   datetime64[ns]
 1   Open          5020 non-null   float64       
 2   High          5020 non-null   float64       
 3   Low           5020 non-null   float64       
 4   Close         5020 non-null   float64       
 5   Adj Close     5020 non-null   float64       
 6   Volume        5020 non-null   int64         
 7   Dividends     5020 non-null   float64       
 8   Stock Splits  5020 non-null   float64       
 9   daily_return  5019 non-null   float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 392.3 KB


In [103]:
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'], errors='coerce')
daily_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           5 non-null      datetime64[ns]
 1   avg_sentiment  5 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 208.0 bytes


In [104]:
merged_data = pd.merge(stock_data, daily_sentiment, left_on='Date', right_on='date', how='left')

In [105]:
merged_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,daily_return,date,avg_sentiment
0,2004-08-19,2.490664,2.591785,2.390042,2.499133,2.496292,897427216,0.0,0.0,NaN,NaT,NaN
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.694573,458857488,0.0,0.0,7.942989,NaT,NaN
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.721690,366857939,0.0,0.0,1.006362,NaT,NaN
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.608991,306396159,0.0,0.0,-4.140766,NaT,NaN
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.637103,184645512,0.0,0.0,1.077510,NaT,NaN


In [106]:
merged_data['avg_sentiment'].fillna(0, inplace=True)
merged_data.dropna(subset=['daily_return'], inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_9188\1834418008.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['avg_sentiment'].fillna(0, inplace=True)


In [113]:
merged_data

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,daily_return,date,avg_sentiment
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.694573,458857488,0.0,0.0,7.942989,NaT,0.0
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.721690,366857939,0.0,0.0,1.006362,NaT,0.0
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.608991,306396159,0.0,0.0,-4.140766,NaT,0.0
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.637103,184645512,0.0,0.0,1.077510,NaT,0.0
5,2004-08-26,2.613952,2.688672,2.606729,2.687676,2.684621,142572401,0.0,0.0,1.801895,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5015,2024-07-24,175.389999,177.949997,173.570007,174.369995,174.369995,31250700,0.0,0.0,-5.027239,NaT,0.0
5016,2024-07-25,174.250000,175.199997,169.050003,169.160004,169.160004,28967900,0.0,0.0,-2.987894,NaT,0.0
5017,2024-07-26,168.770004,169.839996,165.865005,168.679993,168.679993,25150100,0.0,0.0,-0.283762,NaT,0.0
5018,2024-07-29,170.500000,172.160004,169.720001,171.130005,171.130005,13768900,0.0,0.0,1.452462,NaT,0.0


In [108]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5019 entries, 1 to 5019
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           5019 non-null   datetime64[ns]
 1   Open           5019 non-null   float64       
 2   High           5019 non-null   float64       
 3   Low            5019 non-null   float64       
 4   Close          5019 non-null   float64       
 5   Adj Close      5019 non-null   float64       
 6   Volume         5019 non-null   int64         
 7   Dividends      5019 non-null   float64       
 8   Stock Splits   5019 non-null   float64       
 9   daily_return   5019 non-null   float64       
 10  date           5 non-null      datetime64[ns]
 11  avg_sentiment  5019 non-null   float64       
dtypes: datetime64[ns](2), float64(9), int64(1)
memory usage: 509.7 KB


In [109]:
# Verify date alignment
print("Stock Dates Range:", stock_data['Date'].min(), "to", stock_data['Date'].max())
print("News Dates Range:", daily_sentiment['date'].min(), "to", daily_sentiment['date'].max())

# Check for common dates
common_dates = set(stock_data['Date']).intersection(set(daily_sentiment['date']))
print("Common Dates Count:", len(common_dates))

# Check for missing dates in the news data
missing_dates = set(stock_data['Date']) - set(daily_sentiment['date'])
print("Missing Dates in News Data:", len(missing_dates))


Stock Dates Range: 2004-08-19 00:00:00 to 2024-07-30 00:00:00
News Dates Range: 2020-06-04 00:00:00 to 2020-06-10 00:00:00
Common Dates Count: 5
Missing Dates in News Data: 5015
